In [1]:
from network.srcnn import networkTrain

import numpy
import math
import cv2

import os
import h5py
import time

Using TensorFlow backend.


In [2]:
EPOCHS = 1
BATCH_SIZE = 128

scale = 2
FILENAME_TRAIN = "dataTrain_" + scale + ".h5"
FILENAME_VALIDATE = "dataValidate_" + scale + ".h5"
FILENAME_CHECKPOINT = "dataCheckPoint_" + scale + ".h5"

In [3]:
def h5DataRead(file):
    with h5py.File(file, 'r') as hf:
        data = numpy.array(hf.get('data'))
        label = numpy.array(hf.get('label'))
        train_data = numpy.transpose(data, (0, 2, 3, 1))
        train_label = numpy.transpose(label, (0, 2, 3, 1))
        return train_data, train_label

In [4]:
tick1 = cv2.getTickCount()

In [6]:
srcnn_model = networkTrain()
print(srcnn_model.summary())
data, label = h5DataRead(FILENAME_TRAIN)
val_data, val_label = h5DataRead(FILENAME_VALIDATE)

checkpoint = ModelCheckpoint(FILENAME_CHECKPOINT, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='min')

callbacks_list = [checkpoint]

srcnn_model.fit(data, label, batch_size=BATCH_SIZE, validation_data=(val_data, val_label),
                callbacks=callbacks_list, shuffle=True, epochs=EPOCHS, verbose=1)

/home/jovyan/work/network/srcnn.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", input_shape=(32, 32, 1..., kernel_size=(9, 9), filters=128, padding="valid", kernel_initializer="glorot_uniform", use_bias=True)`
  activation='relu', border_mode='valid', bias=True, input_shape=(32, 32, 1)))
/home/jovyan/work/network/srcnn.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", kernel_size=(3, 3), filters=64, padding="same", kernel_initializer="glorot_uniform", use_bias=True)`
  activation='relu', border_mode='same', bias=True))
/home/jovyan/work/network/srcnn.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="linear", kernel_size=(5, 5), filters=1, padding="valid", kernel_initializer="glorot_uniform", use_bias=True)`
  activation='linear', border_mode='valid', bias=True))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 24, 24, 128)       10496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 20, 20, 1)         1601      
Total params: 85,889
Trainable params: 85,889
Non-trainable params: 0
_________________________________________________________________
None
Train on 22000 samples, validate on 2130 samples
Epoch 1/1
22000/22000 [==============================] - 765s 35ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 3.9291e-05 - val_mean_squared_error: 3.9296e-05


In [7]:
score = srcnn_model.evaluate(val_data, val_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 3.92908278145e-05
Test accuracy: 3.92906744147e-05


In [8]:
fileName = 'yayoi_' + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.h5'
srcnn_model.save_weights(fileName)
print(fileName + ' saved')

yayoi_2018-07-09_025600.h5 saved


In [9]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())
print("processed time: " + str(tick) + " ms")

processed time: 885360 ms
